In [1]:
with open('data/movie-actor/actors.csv', encoding='utf-8') as f:
    for _ in range(5):
        print(next(f).strip())

id	k_name	e_name
1	알리야	Aaliyah
2	압바스 키아로스타미	Abbas Kiarostami
3	아벨 페라라	Abel Ferrara
4	아담 아킨	Adam Arkin


In [2]:
with open('data/movie-actor/movies.csv', encoding='utf-8') as f:
    for _ in range(5):
        print(next(f).strip())

movie_id	title	e_title	grade	year	countries
10001	시네마 천국	Cinema Paradiso, 1988	전체 관람가	1990	프랑스 이탈리아
10002	빽 투 더 퓨쳐	Back To The Future, 1985	12세 관람가	1987	미국
10003	빽 투 더 퓨쳐 2	Back To The Future Part 2, 1989	12세 관람가	1990	미국
10004	빽 투 더 퓨쳐 3	Back To The Future Part III, 1990	전체 관람가	1990	미국


In [3]:
with open('data/movie-actor/casting.txt', encoding='utf-8') as f:
    for _ in range(5):
        print(next(f).strip())

movie_idx	actors
22682	7995 4009
25882	154441 12571
133650	312131
29394	6852 96763 3107 7067 39104


In [4]:
# pre defined casting weight graph
# see more data/movie-actor/make_casting_graph.py

import pickle
from pprint import pprint 

with open('data/movie-actor/casting_graph.pkl', 'rb') as f:
    graph = pickle.load(f)

# casting weight of movie = 10001
pprint(sorted(graph['10001'].items(), key=lambda x:-x[1]))

[('4374', 0.1711229946524064),
 ('178', 0.15040106951871657),
 ('3241', 0.13101604278074866),
 ('47952', 0.11296791443850267),
 ('47953', 0.0962566844919786),
 ('19538', 0.08088235294117647),
 ('18991', 0.06684491978609626),
 ('47954', 0.05414438502673797),
 ('6038', 0.0427807486631016),
 ('24102', 0.032754010695187165),
 ('47955', 0.02406417112299465),
 ('16903', 0.016711229946524065),
 ('47956', 0.0106951871657754),
 ('47957', 0.006016042780748663),
 ('47958', 0.00267379679144385),
 ('47959', 0.0006684491978609625)]


In [5]:
# create idx to actor name function
with open('data/movie-actor/actors.csv', encoding='utf-8') as f:
    next(f)
    docs = [line[:-1].split('\t') for line in f]
    # English name if exist else Korean name
    _idx2actor = {doc[0]:(doc[2] if doc[2] else doc[1]) for doc in docs}

idx2actor = lambda idx: _idx2actor.get(idx, 'Unknown')

In [6]:
# create idx to movie name function
def append_year_countries(year, countries):
    if year and countries:
        return ' ({}, {})'.format(year, countries)
    elif year:
        return ' ({})'.format(year)
    elif countries:
        return ' ({})'.format(countries)
    return ''

with open('data/movie-actor/movies.csv', encoding='utf-8') as f:
    next(f)
    docs = [line[:-1].split('\t') for line in f]
    _idx2movie = {doc[0]:'{}{}'.format(doc[1], append_year_countries(doc[4], doc[5])) for doc in docs if len(docs)}

idx2movie = lambda idx: _idx2movie.get(idx, 'Unknown')

In [7]:
for actor, weight in graph['134963'].items():
    print('actor = {:7}, weight = {}'.format(actor, weight))

actor = 5751   , weight = 0.20634920634920634
actor = 135256 , weight = 0.17582417582417584
actor = 4651   , weight = 0.14774114774114774
actor = 97297  , weight = 0.1221001221001221
actor = 347222 , weight = 0.0989010989010989
actor = 66630  , weight = 0.07814407814407814
actor = 7269   , weight = 0.05982905982905983
actor = 28674  , weight = 0.04395604395604396
actor = 353879 , weight = 0.030525030525030524
actor = 19339  , weight = 0.019536019536019536
actor = 197878 , weight = 0.01098901098901099
actor = 357382 , weight = 0.004884004884004884
actor = 1559   , weight = 0.001221001221001221


In [8]:
# test
print(idx2movie('134963'), end='\n\n')
for actor, weight in sorted(graph['134963'].items(), key=lambda x:-x[1]):
    print('{} : {}'.format(idx2actor(actor), weight))

print('\n5751 = {}'.format(idx2actor('5751')))

라라랜드 (2016, 미국)

Ryan Gosling : 0.20634920634920634
Emma Stone : 0.17582417582417584
J.K. Simmons : 0.14774114774114774
John Legend : 0.1221001221001221
Sonoya Mizuno : 0.0989010989010989
Rosemarie DeWitt : 0.07814407814407814
Jason Fuchs : 0.05982905982905983
Meagen Fay : 0.04395604395604396
Jessica Rothe : 0.030525030525030524
Miles Anderson : 0.019536019536019536
Trevor Lissauer : 0.01098901098901099
Callie Hernandez : 0.004884004884004884
Tom Everett Scott : 0.001221001221001221

5751 = Ryan Gosling


In [9]:
# bi-directed graph
# graph has only one-way link: movie -> actor
actor_weight_sum = {}

# cumulate actor weights
for movie, actors in graph.items():
    for actor, weight in actors.items():
        actor_weight_sum[actor] = actor_weight_sum.get(actor, 0) + weight

# make bi-directed graph
from collections import defaultdict
g = defaultdict(lambda: {})
for movie, actors in graph.items():
    g['movie {}'.format(movie)] = {'actor {}'.format(a):w for a,w in actors.items()}
    for actor, weight in actors.items():
        g['actor {}'.format(actor)]['movie {}'.format(movie)] = weight / actor_weight_sum[actor]

g = dict(g)

In [10]:
g['movie 134963']

{'actor 135256': 0.17582417582417584,
 'actor 1559': 0.001221001221001221,
 'actor 19339': 0.019536019536019536,
 'actor 197878': 0.01098901098901099,
 'actor 28674': 0.04395604395604396,
 'actor 347222': 0.0989010989010989,
 'actor 353879': 0.030525030525030524,
 'actor 357382': 0.004884004884004884,
 'actor 4651': 0.14774114774114774,
 'actor 5751': 0.20634920634920634,
 'actor 66630': 0.07814407814407814,
 'actor 7269': 0.05982905982905983,
 'actor 97297': 0.1221001221001221}

In [11]:
for movie in g['actor 5751']:
    movie_idx = movie.split()[1]
    print(idx2movie(movie_idx))

라라랜드 (2016, 미국)
하프 넬슨 (2012, 미국)
블루 발렌타인 (2012, 미국)
갱스터 스쿼드 (미국)
드라이브 (2011, 미국)
테렌스 맬릭 프로젝트(가제) (미국)
블레이드 러너 2049 (미국 캐나다 영국)
프랙처 (미국)
빅쇼트 (2016, 미국)
크레이지, 스투피드, 러브 (미국)
로건스 런 (미국)
내겐 너무 사랑스러운 그녀 (2008, 미국)
더 슬로터 룰 (미국)
유나이티드 스테이츠 오브 리랜드 (미국)
올 굿 에브리씽 (미국)
머더 바이 넘버 (2002, 미국)
킹메이커 (2012, 미국)
노트북 (2004, 미국)
온리 갓 포기브스 (2014, 프랑스 태국 미국 스웨덴 덴마크)
플레이스 비욘드 더 파인즈 (2013, 미국)
빌리버 (미국)
스테이 (2005, 미국)
마이 라이프 디렉티드 바이 니콜라스 윈딩 레픈 (덴마크)
리제너레이션 (미국)
나이스 가이즈 (2016, 미국)
플래시 포워드 (미국 캐나다)
MMC (미국)
프랑켄슈타인과 나 (캐나다)


In [13]:
from pagerank import pagerank

rank = pagerank(g,
                bias=None,
                df=0.15,
                max_iter=50,
                converge_error=0.0001,
                verbose=1)

Iteration = 1, diff = 0.7663714375352587
Iteration = 2, diff = 0.5973778498022786
Iteration = 3, diff = 0.4753779904088874
Iteration = 4, diff = 0.3876545641819942
Iteration = 5, diff = 0.3201995967978161
Iteration = 6, diff = 0.26685630373296704
Iteration = 7, diff = 0.2236944600745994
Iteration = 8, diff = 0.18813011706126767
Iteration = 9, diff = 0.15857142935860843
Iteration = 10, diff = 0.13385368096602895
Iteration = 11, diff = 0.11311713940222842
Iteration = 12, diff = 0.09567336023364868
Iteration = 13, diff = 0.08097322685291032
Iteration = 14, diff = 0.06857008757046942
Iteration = 15, diff = 0.05809138788591349
Iteration = 16, diff = 0.04923210998635747
Iteration = 17, diff = 0.041735647433191095
Iteration = 18, diff = 0.035389856617322565
Iteration = 19, diff = 0.030015020263829822
Iteration = 20, diff = 0.02546128515350866
Iteration = 21, diff = 0.021601837840377395
Iteration = 22, diff = 0.01833011710695322
Iteration = 23, diff = 0.015555814776266413
Iteration = 24, diff 

In [15]:
movie_rank = {node:rank for node, rank in rank.items() if node[0] == 'm'}
actor_rank = {node:rank for node, rank in rank.items() if node[0] == 'a'}

In [22]:
list(movie_rank)[:10]

['movie 94969',
 'movie 121413',
 'movie 71232',
 'movie 11446',
 'movie 62007',
 'movie 86098',
 'movie 10101',
 'movie 86022',
 'movie 108745',
 'movie 108315']

In [24]:
# top rank movie
# filtering Korean movie
korean_movies = {movie:weight for movie, weight in movie_rank.items() if '한국)' in idx2movie(movie.split()[1])}

# movies starring many famous actors
for movie, _ in sorted(korean_movies.items(), key=lambda x:-x[1])[:100]:
    movie_idx = movie.split()[1]
    print(idx2movie(movie_idx))

개그콘서트 (1999, 한국)
폭소 클럽 (한국)
깡패와 건달로 본 한국 100년 (한국)
티라노의 발톱 (1994, 한국)
살인의 강 (2010, 한국)
신세대 보고서 어른들은 몰라요 (한국)
박하사탕 (2000, 한국)
기담전설 2 - 소름 (2009, 한국)
나와 함께 블루스를 (한국)
시간의 춤 (2009, 한국)
우리 학교 (2007, 한국)
그 사람 추기경 (2014, 한국)
별별 이야기 2 - 여섯 빛깔 무지개 (2008, 한국)
목포는 항구다 (2004, 한국)
실미도 (2003, 한국)
웃찾사 (한국)
레볼루션 (한국)
원 나잇 스탠드 (2010, 한국)
전설의 고향 1997 (한국)
우린 액션배우다 (2008, 한국)
반드시 크게 들을 것 (2010, 한국)
누구에게나 찬란한 (2014, 한국)
이리 (2008, 한국)
아 유 레디? (2002, 한국)
마이너클럽 (한국)
코미디에 빠지다 (2012, 한국)
신혼여행 (2000, 한국)
JSA 남북공동초등학교 (한국)
봉자 (2000, 한국)
작은 연못 (2010, 한국)
자명고 (2009, 한국)
토지 (2004, 한국)
주몽 (2006, 한국)
서울역 (2016, 한국)
코미디쇼 웃으면 복이 와요 (한국)
진짜 사나이 (1996, 한국)
섹스 볼란티어 (2010, 한국)
얼짱시대_시즌6 (한국)
자카르타 (2000, 한국)
이카루스 (한국)
쌀 (한국)
똥례... 하면 죽는다 (2015, 한국)
모범생 (한국)
할머니의 먼 집 (2016, 한국)
다방의 푸른 꿈 (2017, 한국)
바세코의 아이들 (2014, 한국)
전설의 고향 1996 (한국)
개그야 (한국)
유관순 (한국)
휘파람 공주 (2002, 한국)
학교종을 울려라 (한국)
레디액션 청춘 (2014, 한국)
제5의 사나이 (1991, 한국)
부활의 노래 (1991, 한국)
이웃집 좀비 (2010, 한국)
어허 어이 어이가리 (한국)
바리바리 짱 (2005, 한국)
흔들리는 물결 (2016, 한국)
용병 이반 (1997, 한국)
인

In [26]:
# top actor
for actor, _ in sorted(actor_rank.items(), key=lambda x:-x[1])[:100]:
    actor_idx = actor.split()[1]
    print(idx2actor(actor_idx), end=' ')

Mel Blanc 김지미 최무룡 신영균 강신성일 윤정희 박노식 문희 문정숙 김진규 장동휘 남정임 김승호 엄앵란 남궁원 최은희 최지희 김혜정 고은아 오영일 도금봉 허장강 김희갑 조미령 黃貞順 이대엽 김동원 황해 태현실 김석훈 이민자 남진 Choi Nam-Hyun 이예춘 전계현 방성자 구봉서 주증녀 강문 김희라 박암 Arthur Q. Bryan 한은진 Winston Hibler 주연 김정훈 이경희 안인숙 Barbara Hammer 김지수 독고성 이수련 백영민 서영춘 정애란 곽규석 박종화 이민 June Foray 윤인자 김성옥 이순재 윤일봉 최난경 오지명 김창숙 윤소라 강미애 김희준 박지영 양훈 신일룡 신영일 Stan Freberg 하명중 김혜경 전영선 이빈화 전현무 주선태 장혁 김운하 나훈아 방수일 홍세미 한성 김석강 남미리 최인숙 Daws Butler 양미희 David Attenborough 양석천 손미희자 노경희 김의향 남춘역 오유경 박병호 최삼 

유명한 배우나 유명한 영화가 아닌, 여러 영화에 자주 출연한 배우들의 랭킹이 높고, 그런 배우들이 많이 출연한 영화들이 랭킹이 높음. 그렇기 때문에 80 ~ 90 년대의 중국영화 배우나 한국영화 배우들이 높은 ranking 을 보입니다. 주로 한국영화는 80 ~ 90 년대에 몇몇 배우들에 의하여 다작이 이뤄졌기 때문입니다. 대표적으로 이순재, 신영일 등이 있으며, 이순재 선생님이 출연한 작품은 네이버 영화 데이터베이스 기준 216 편입니다. 

bias 를 설정하여 personalized PageRank 로 이용해야 유명한 영화나 ticket power 가 높은 배우를 찾을 수 있습니다. 